In [1]:
from tsa.models import Arima

In [2]:
import pandas as pd

In [3]:
import sys

original_stdout = sys.stdout 
	
def outfile(x):
    with open('output.txt', 'w') as f:
        sys.stdout = f
        print(x)
        sys.stdout = original_stdout 

In [4]:
df = pd.read_csv('datasets/5/kalimati_tarkari_dataset.csv.zip')
df = pd.DataFrame(df[df['Commodity'] == 'Potato White'])
df

,SN,Commodity,Date,Unit,Minimum,Maximum,Average
3,3,Potato White,2013-06-16,Kg,15.0,16.0,15.5
77,77,Potato White,2013-06-17,Kg,15.0,16.0,15.5
150,150,Potato White,2013-06-18,Kg,15.0,16.0,15.5
223,223,Potato White,2013-06-19,Kg,15.0,16.0,15.5
296,296,Potato White,2013-06-20,Kg,15.0,16.0,15.5
...,...,...,...,...,...,...,...
195878,195878,Potato White,2021-04-29,Kg,30.0,32.0,31.0
196126,196126,Potato White,2021-05-02,Kg,35.0,40.0,37.5
196372,196372,Potato White,2021-05-05,Kg,30.0,35.0,32.5
196459,196459,Potato White,2021-05-06,Kg,30.0,35.0,32.5


In [5]:
from tsa.dateconverter import DateConverter
from tsa.interpolation import interpolate_dates as interpolate

converter = DateConverter()

In [6]:
df['Date'] = df['Date'].apply(lambda x: converter.convert_date(x, infer_format=True))
df = interpolate(df, 'Date', ['Average'])
df

,Average,Date
0,15.500000,2013-06-16
1,15.500000,2013-06-17
2,15.500000,2013-06-18
3,15.500000,2013-06-19
4,15.500000,2013-06-20
...,...,...
2878,35.833333,2021-05-03
2879,34.166667,2021-05-04
2880,32.500000,2021-05-05
2881,32.500000,2021-05-06


In [7]:
y = 'Average'

In [8]:
df.isna().sum()

Average    0
Date       0
dtype: int64

In [9]:
train_df = df[:int(0.8 * len(df))]
test_df = df[int(0.8 * len(df)):]

In [10]:
df = train_df

In [11]:
a = Arima()

In [13]:
a(df, 'Date', y)

In [15]:
a.test_stationarity(1) # d= 1

1. ADF :  -14.945933101879671
2. P-Value :  1.3016350519391587e-27
3. Num Of Lags :  10
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 2294
5. Critical Values :
	 1% :  -3.4332038066451385
	 5% :  -2.8628007468583427
	 10% :  -2.5674411527893164


As p-value is inside the confidence interval of 95%, series is stationary.
Differenced: 1


In [19]:
outfile(a.acf_plot(1)['lower']) # 1

In [23]:
outfile(a.pacf_plot(1)['lower'])

In [24]:
a.fit((2,1,1))

                               SARIMAX Results                                
Dep. Variable:                Average   No. Observations:                 2306
Model:                 ARIMA(2, 1, 1)   Log Likelihood               -4399.269
Date:                Mon, 03 Jul 2023   AIC                           8806.538
Time:                        16:29:53   BIC                           8829.510
Sample:                    06-16-2013   HQIC                          8814.913
                         - 10-08-2019                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.3352      0.108     -3.103      0.002      -0.547      -0.123
ar.L2         -0.1264      0.032     -3.951      0.000      -0.189      -0.064
ma.L1          0.0386      0.109      0.355      0.7

/home/shail/bits/PS1/time_series_analysis/env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/shail/bits/PS1/time_series_analysis/env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/shail/bits/PS1/time_series_analysis/env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [25]:
a.error_metrics(test_df, y)

,Metric,Value
0,MAPE,0.505575
1,MAE,16.858608
2,MSE,395.347640


In [26]:
params = a.plot_forecast(test_df, y)

In [28]:
outfile(params['y_series'])